## Machine Learning

### Importación de las librerías necesarias

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Se utilizarán las librerías sklearn.feature_extraction.text y sklearn.metrics.pairwise para implementar un sistema de recomendación basado en filtrado de contenido en el que se compararán las similitudes entre diferentes elementos de un conjunto de datos. La librería CountVectorizer se utilizará para convertir los datos de texto en una matriz numérica, mientras que cosine_similarity se utilizará para calcular la similitud coseno entre los elementos de la matriz. Esto permitirá generar recomendaciones personalizadas basadas en los patrones de similitud de los datos de entrada.

### Carga de datos

In [3]:
df_movie=pd.read_csv(r"Datasets/movies_transformation_ML.csv")
df_movie.head(3)

,title,label
0,the grand seduction,movi don mckellar brendan gleeson taylor kitsc...
1,take care good night,movi girish joshi mahesh manjrekar abhay mahaj...
2,secrets of deception,movi josh webber tom sizemor lorenzo lama robe...


### Matriz de caracaterísticas

In [42]:
cv=CountVectorizer(max_features=1000,stop_words="english")
vector=cv.fit_transform(df_movie.label).toarray()

* Al utilizar max_features=1000 en cv=CountVectorizer(max_features=1000, stop_words="english"), estoy limitando el número máximo de palabras que se utilizarán para generar la matriz de características. Esto me permite trabajar con un conjunto de características más manejable, lo que puede reducir el tiempo de procesamiento y aumentar la precisión del modelo. También estoy eliminando las palabras comunes en inglés utilizando stop_words="english", lo que puede mejorar la calidad de las características restantes y reducir el ruido en el modelo.

In [69]:
# Visualizando el array vector
print(vector.shape)
vector

(22998, 1000)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int64)

### Matriz de Similitud

In [43]:
similarity_score=cosine_similarity(vector)

In [70]:
# Visualizando el array similarity_score
print(similarity_score.shape)
similarity_score

(22998, 22998)


array([[1.        , 0.13867505, 0.11020775, ..., 0.13977131, 0.13074409,
        0.10529963],
       [0.13867505, 1.        , 0.19867985, ..., 0.18898224, 0.23570226,
        0.3322053 ],
       [0.11020775, 0.19867985, 1.        , ..., 0.25031309, 0.23414645,
        0.26401   ],
       ...,
       [0.13977131, 0.18898224, 0.25031309, ..., 1.        , 0.35634832,
        0.179374  ],
       [0.13074409, 0.23570226, 0.23414645, ..., 0.35634832, 1.        ,
        0.20134682],
       [0.10529963, 0.3322053 , 0.26401   , ..., 0.179374  , 0.20134682,
        1.        ]])

* Crearemos una función que se utilizará para indexar las películas más similares a una película específica en el conjunto de datos. Dado un título de película, la función encuentra su índice correspondiente en el conjunto de datos y utiliza la matriz de similitud previamente calculada, para determinar los 5 elementos más similares a la película de referencia. La función devuelve una lista de índices correspondientes a las películas más similares en orden descendente, lo que permite utilizar esta información para hacer recomendaciones personalizadas en el endpoint 7 de la API de manera más eficiente y efectiva.

In [73]:
def index_movie(titulo:str):
    index=df_movie[df_movie.title==titulo].index[0]
    score=list(enumerate(similarity_score[index]))
    score=sorted(score,key=lambda x: x[1],reverse=True)
    # El índice 0 siempre corresponde a la misma película, por lo cual, no se considera
    score=score[1:6]
    return [i[0] for i in score]

In [77]:
df_movie["index_movie"]=df_movie.title.apply(index_movie)
df_movie

,title,label,index_movie
0,the grand seduction,movi don mckellar brendan gleeson taylor kitsc...,"[4800, 19930, 19589, 7248, 2638]"
1,take care good night,movi girish joshi mahesh manjrekar abhay mahaj...,"[7260, 12730, 2603, 7879, 7463]"
2,secrets of deception,movi josh webber tom sizemor lorenzo lama robe...,"[13308, 17119, 13031, 11029, 3689]"
3,pink: staying true,movi sonia anderson interview with: pink adel ...,"[14308, 13175, 13275, 22078, 18491]"
4,monster maker,movi gile foster harri dean stanton kieran o'b...,"[13371, 7866, 7559, 6157, 6497]"
...,...,...,...
22993,zodiac,movi david fincher mark ruffalo jake gyllenha ...,"[19892, 22059, 11402, 20148, 22262]"
22994,zombie dumb,tv show 7+ kids' tv korean tv show tv comedi w...,"[8708, 8724, 8726, 8728, 8752]"
22995,zombieland,movi ruben fleischer jess eisenberg woodi harr...,"[13028, 21899, 21137, 21454, 21836]"
22996,zoom,movi peter hewitt tim allen courteney cox chev...,"[20671, 17529, 18251, 20427, 10612]"


#### Exportación del `df_movie` para la realización del endpoint de ML (función 7)

In [80]:
# La columna "label" ya no será necesaria
df_movie.drop(columns=["label"]).to_csv("movies_ML_API.csv",index=False)

In [10]:
df=pd.read_csv(r"Datasets/movies_ML_API.csv")
df

,title,index_movie
0,the grand seduction,"[4800, 19930, 19589, 7248, 2638]"
1,take care good night,"[7260, 12730, 2603, 7879, 7463]"
2,secrets of deception,"[13308, 17119, 13031, 11029, 3689]"
3,pink: staying true,"[14308, 13175, 13275, 22078, 18491]"
4,monster maker,"[13371, 7866, 7559, 6157, 6497]"
...,...,...
22993,zodiac,"[19892, 22059, 11402, 20148, 22262]"
22994,zombie dumb,"[8708, 8724, 8726, 8728, 8752]"
22995,zombieland,"[13028, 21899, 21137, 21454, 21836]"
22996,zoom,"[20671, 17529, 18251, 20427, 10612]"
